In [1]:
import boto3
import json

bedrock_client = boto3.client('bedrock-runtime')
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

user_input ="Hola"

messages = [{"role": "user","content": [{"type":"text", "text": user_input}]}]
system_prompt = "You are a helpful assistant."

body=json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "system": system_prompt,
        "messages": messages
    }  
)  

response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body
)

In [2]:
response.get('output').get('message').get('content')[0].get('text')

AttributeError: 'NoneType' object has no attribute 'get'

In [5]:
response_body = json.loads(response.get('body').read())

In [6]:
response_body.get('content')[0].get('text')

'¡Hola! ¿Cómo estás?'

response_body

# Tool usage

In [13]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

bedrock_client = boto3.client('bedrock-runtime')
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

def confirm_check_in(locator, last_name):
    print (locator, last_name)
    return "check-in confirmado"

In [9]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "confirm_check_in",
                "description": "Confirma el checkin del pasajero usando record locator y apellido",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "locator": {
                                "type": "string",
                                "description": "El record locator en formato ABC123 (tres letras y tres dígitos)",
                            },
                            "last_name": {
                                "type": "string",
                                "description": "El apellido del pasajero como Rodríguez, Vásquez o Garrido",
                            },
                        },
                        "required": ["locator", "last_name"],
                    }
                },
            }
        }
    ]
}

In [17]:
def generate_text(bedrock_client, model_id, tool_config, input_text):
    logger.info("Generating text with model %s", model_id)

    messages = [{"role": "user","content": [{"text": input_text}]}]

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    output_message = response['output']['message']
    messages.append(output_message)
    stop_reason = response['stopReason']

    if stop_reason == 'tool_use':
        # Tool use requested. Call the tool and send the result to the model.
        tool_requests = response['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                logger.info("Requesting tool %s. Request: %s",
                            tool['name'], tool['toolUseId'])

                if tool['name'] == 'confirm_check_in':
                    tool_result = {}

                    res = confirm_check_in(tool['input']['locator'], tool['input']['last_name'])
                    tool_result = {
                        "toolUseId": tool['toolUseId'],
                        "content": [{"json": {"result": res}}]
                    }
      

                    tool_result_message = {
                        "role": "user",
                        "content": [
                            {
                                "toolResult": tool_result

                            }
                        ]
                    }
                    messages.append(tool_result_message)

                    # Send the tool result to the model.
                    response = bedrock_client.converse(
                        modelId=model_id,
                        messages=messages,
                        toolConfig=tool_config
                    )
                    output_message = response['output']['message']
                    print(f"output_message: {output_message}")

    # print the final response from the model.
    for content in output_message['content']:
        print(f"{content['text']}")

In [18]:
input_text = "Hola mi me llamo enrique rodriguez y mi codigo de reserva es ABC222"


In [19]:
generate_text(bedrock_client, model_id, tool_config, input_text)


INFO:__main__:Generating text with model anthropic.claude-3-haiku-20240307-v1:0
INFO:__main__:Requesting tool confirm_check_in. Request: tooluse_MSB8GEM1RKKpKRMvwc9CbQ


ABC222 Rodriguez
output_message: {'role': 'assistant', 'content': [{'text': 'Gracias por proporcionar la información. He confirmado su registro de facturación con el código de reserva ABC222 y el apellido Rodriguez. Disfrute de su vuelo.'}]}
Gracias por proporcionar la información. He confirmado su registro de facturación con el código de reserva ABC222 y el apellido Rodriguez. Disfrute de su vuelo.
